# Tweets Data

In [4]:
import pandas as pd
import numpy as np


In [5]:
def info(ticker):
    """show the basic info of a company
    """
    df_ticker = pd.read_csv(ticker+"_tweets.csv")
    df_ticker.info()

info("MSFT")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8798 entries, 0 to 8797
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   8626 non-null   float64
 1   date         8798 non-null   object 
 2   followers    8798 non-null   object 
 3   screen_name  8798 non-null   object 
 4   tweet        8798 non-null   object 
 5   tweet_id     8798 non-null   object 
dtypes: float64(1), object(5)
memory usage: 412.5+ KB


In [7]:
df = pd.read_csv("MSFT_tweets.csv")
df = df.dropna()    #remove the rows with na values
df = df.reset_index()   # reset the index
df = df.loc[:, ['date', 'followers', 'tweet']]    # only take the key info we'll use
df

,date,followers,tweet
0,2020-10-08 23:43:25,3308.0,#Free Trial available at https://t.co/x2kmaKA...
1,2020-10-08 23:42:13,3308.0,4 hour charts for all 78 instruments are avail...
2,2020-10-09 01:57:58,910.0,Azure Communication Services enables developer...
3,2020-10-09 12:21:46,1471.0,Microsoft is letting employees work from home ...
4,2020-10-09 19:06:02,3308.0,1 hour charts for all 78 instruments are avail...
...,...,...,...
8621,2020-04-02 07:03:04,1398.0,📣💡 TODAY I become a Microsoft Most Valuable Pr...
8622,2020-04-01 11:20:06,1413.0,If you're in need of a #CRM system but are uns...
8623,2020-04-01 19:08:49,5.0,New ticket has been created on options trading...
8624,2020-04-01 15:58:51,5.0,New ticket has been created on options trading...


In [8]:
df['date']=pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')    # transfer the type of date
df['followers'] = df['followers'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8626 entries, 0 to 8625
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       8626 non-null   datetime64[ns]
 1   followers  8626 non-null   float64       
 2   tweet      8626 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 202.3+ KB


In [9]:
df['date'] = df['date'].dt.normalize()    # we only need the date
df

,date,followers,tweet
0,2020-10-08,3308.0,#Free Trial available at https://t.co/x2kmaKA...
1,2020-10-08,3308.0,4 hour charts for all 78 instruments are avail...
2,2020-10-09,910.0,Azure Communication Services enables developer...
3,2020-10-09,1471.0,Microsoft is letting employees work from home ...
4,2020-10-09,3308.0,1 hour charts for all 78 instruments are avail...
...,...,...,...
8621,2020-04-02,1398.0,📣💡 TODAY I become a Microsoft Most Valuable Pr...
8622,2020-04-01,1413.0,If you're in need of a #CRM system but are uns...
8623,2020-04-01,5.0,New ticket has been created on options trading...
8624,2020-04-01,5.0,New ticket has been created on options trading...


In [10]:
###########################################
#########Analyze the sentiments################
###########################################

In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [12]:
def sentiment_scores(sentence):
    """return a sentiment score of the input sentence
    """
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    return sentiment_dict['compound']

In [13]:
#df1 = df.head(10).copy()
df['senti_score'] = 0 

for i in range(0, len(df)):
    df.loc[i, 'senti_score'] = sentiment_scores(df.loc[i, 'tweet'])              

In [14]:
df1 = df.copy()
df1

,date,followers,tweet,senti_score
0,2020-10-08,3308.0,#Free Trial available at https://t.co/x2kmaKA...,0.5106
1,2020-10-08,3308.0,4 hour charts for all 78 instruments are avail...,0.0000
2,2020-10-09,910.0,Azure Communication Services enables developer...,0.4019
3,2020-10-09,1471.0,Microsoft is letting employees work from home ...,0.0000
4,2020-10-09,3308.0,1 hour charts for all 78 instruments are avail...,0.0000
...,...,...,...,...
8621,2020-04-02,1398.0,📣💡 TODAY I become a Microsoft Most Valuable Pr...,0.8620
8622,2020-04-01,1413.0,If you're in need of a #CRM system but are uns...,0.8271
8623,2020-04-01,5.0,New ticket has been created on options trading...,0.4359
8624,2020-04-01,5.0,New ticket has been created on options trading...,0.4359


In [15]:
def weighted_avg(group, avg_name, weight_name):
    """ calculate weighted avg
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [86]:
senti_series = df1.groupby("date").apply(weighted_avg, "senti_score", "followers")

In [87]:
type(senti_series)

pandas.core.series.Series

In [88]:
senti_series.to_csv("MSFT_series.csv")

In [89]:
senti_series

date
2020-04-01    0.270791
2020-04-02    0.237726
2020-04-03    0.115113
2020-04-04    0.217096
2020-04-05    0.438362
                ...   
2020-10-06    0.805220
2020-10-07    0.181335
2020-10-08    0.639552
2020-10-09    0.237814
2020-10-12    0.680800
Length: 193, dtype: float64

In [90]:
senti = pd.DataFrame({'tweets_date':senti_series.index, 'score':senti_series.values})

In [91]:
senti['day'] = senti['tweets_date'].dt.dayofweek

In [92]:
senti = senti.set_index("tweets_date")

In [93]:
senti.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 193 entries, 2020-04-01 to 2020-10-12
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   score   193 non-null    float64
 1   day     193 non-null    int64  
dtypes: float64(1), int64(1)
memory usage: 4.5 KB


In [102]:
senti

,score,day
tweets_date,,
2020-04-01,0.270791,2
2020-04-02,0.237726,3
2020-04-03,0.115113,4
2020-04-04,0.217096,5
2020-04-05,0.438362,6
...,...,...
2020-10-06,0.805220,1
2020-10-07,0.181335,2
2020-10-08,0.639552,3


# Stock Market Data

In [95]:
stock = pd.read_csv("MSFT_stock.csv")
stock["Movement"] = stock["Adj Close"].pct_change() *100
stock = stock.loc[:, ['Date', 'Movement']]
stock

,Date,Movement
0,2020-04-01,NaN
1,2020-04-02,2.070864
2,2020-04-03,-0.921028
3,2020-04-06,7.436775
4,2020-04-07,-1.077021
...,...,...
121,2020-09-23,-3.292837
122,2020-09-24,1.296179
123,2020-09-25,2.278658
124,2020-09-28,0.779518


In [96]:
stock['Date']=pd.to_datetime(stock['Date'], format='%Y-%m-%d') 

In [97]:
import datetime
stock.Date = stock.Date + datetime.timedelta(days = -1)

In [98]:
stock = stock.set_index("Date")

# 合并

In [99]:
result = senti.join(stock, how="inner")

In [100]:
result

,score,day,Movement
2020-04-01,0.270791,2,2.070864
2020-04-02,0.237726,3,-0.921028
2020-04-05,0.438362,6,7.436775
2020-04-06,0.096384,0,-1.077021
2020-04-07,0.141523,1,1.003119
...,...,...,...
2020-09-22,0.481326,1,-3.292837
2020-09-23,0.275897,2,1.296179
2020-09-24,0.044069,3,2.278658
2020-09-27,0.145333,6,0.779518


In [101]:
result.to_csv("MSFT_final.csv")